In [16]:
%reload_ext autoreload
%autoreload 2

import papermill as pm

In [17]:
notebooks = [
  '01-data-collection.ipynb',
  '02-exploratory-data-analysis.ipynb',
  '03-data-preparation.ipynb',
  '04-modelisation.ipynb'
]
for notebook in notebooks:
    notebook_path = f"../notebooks/{notebook}"
    output_notebook = f"{'../notebooks/outputs'}/{notebook.replace('.ipynb', '_output.ipynb')}"
    pm.execute_notebook(notebook_path, output_notebook)

Executing:   0%|          | 0/34 [00:00<?, ?cell/s]

Executing:   0%|          | 0/44 [00:00<?, ?cell/s]

Executing:   0%|          | 0/47 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [10]":
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[10], line 25
     23 estimator = settings["estimator"]
     24 param_grid = settings["params"]
---> 25 pipeline = define_pipeline(numerical_transformer=[SimpleImputer(strategy="median"), RobustScaler()],
     26                 categorical_transformer=[SimpleImputer(strategy="constant", fill_value="undefined"), OneHotEncoder(drop="if_binary", handle_unknown="ignore")],
     27                 target_transformer=True,
     28                 estimator=estimator
     29             ) 
     30 grid_search = GridSearchCV(
     31     estimator=pipeline,  # Instantiate the estimator
     32     param_grid=param_grid,
   (...)
     36     n_jobs=-1  # Use all available cores
     37 )
     38 grid_s